# Stack Data

In [2]:
import xml.etree.ElementTree as ET
import sys, os
import pandas as pd
import numpy as np

def xml2pd(xml_file):
    ''' Load xml file as Pandas DataFrame'''
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    cols = root[0].attrib.keys()
    rows = [child.attrib for child in root]
    return pd.DataFrame(rows, columns = cols)

## NLP

In [5]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from bs4 import BeautifulSoup
import re, string

In [131]:
stopWords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def removeHTML(text):
    ''' Remove HTML from given text '''
    soup = BeautifulSoup(text, 'lxml')
    return soup.get_text()

def removeURLs(text):
    ''' Remove URLs from given text '''
    return re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", text)

def removePunctuation(text):
    ''' Remove punctuations from given text'''
    return "".join([c for c in text if c not in string.punctuation])

def removeStopWords(text):
    ''' Remove stopwords from given text '''
    return "".join([word for word in text if word not in stopWords])

def removeTags(text):
    ''' Remove code and pre tags from text '''
    soup = BeautifulSoup(text, 'lxml')
    for element in soup.find_all("code", "pre"): 
        element.decompose()
    return soup.get_text()

def lemmatizeText(text):
    ''' Lemmatize the given text '''
    return [lemmatizer.lemmatize(i) for i in text]

In [103]:
posts = xml2pd('Posts.xml')

In [16]:
removePunctuation(posts['Body'][5])

'pIve heard that the quality of a monte carlo ray tracer based on path tracing algorithms is much more realistic than a distributed stochastic engine I try to understand why but Im just at the beginning p\n\npIn order to dive into this topic and understand the basics can someone point me into the right direction What part of the algorithm leads into a more realistic render result p\n'

## Analysis with tokens

In [19]:
dfmini = pd.DataFrame(columns=["body", "points", "postType"])
tokenizer = RegexpTokenizer(r'\w+')

dfmini["body"] = posts['Body'].apply(lambda x: tokenizer.tokenize(x.lower()))
dfmini["points"] = posts['Score']
dfmini["postType"] = posts["PostTypeId"]

# PostTypeId 1 = Question, 2 = Answer
dfmini.head(20)

,body,points,postType
0,"[p, i, saw, a, a, href, https, www, youtube, c...",8,1
1,"[p, the, convention, in, graphics, is, that, p...",11,1
2,"[p, sometimes, i, use, vector, graphics, simpl...",20,1
3,"[p, i, just, implemented, some, interpolated, ...",11,1
4,"[p, i, have, an, opengl, application, which, u...",11,1
5,"[p, i, ve, heard, that, the, quality, of, a, m...",30,1
6,"[p, there, might, be, p, p, less, technical, a...",8,2
7,"[p, a, few, years, ago, i, tried, to, implemen...",17,1
8,"[p, as, thebuzzsaw, said, it, does, depend, on...",15,2
9,"[p, updating, an, area, of, memory, in, the, g...",11,2


## StopWord Removal

In [24]:
dfmini['body'] = dfmini['body'].apply(lambda x: removeStopWords(x))
dfmini['body'].head(20)

0     [p, saw, href, https, www, youtube, com, watch...
1     [p, convention, graphics, performing, fewer, s...
2     [p, sometimes, use, vector, graphics, simply, ...
3     [p, implemented, interpolated, texture, sampli...
4     [p, opengl, application, uses, stencil, tests,...
5     [p, heard, quality, monte, carlo, ray, tracer,...
6     [p, might, p, p, less, technical, answer, p, p...
7     [p, years, ago, tried, implement, href, http, ...
8     [p, thebuzzsaw, said, depend, lots, things, in...
9     [p, updating, area, memory, graphics, device, ...
10    [p, wikipedia, href, https, en, wikipedia, org...
11    [p, strong, distributed, ray, tracing, strong,...
12    [p, really, way, p, p, geometry, shader, invoc...
13    [p, 1978, edwin, catmull, jim, clark, defined,...
14    [blockquote, p, stencil, buffer, contains, per...
15    [p, term, distributed, ray, tracing, originall...
16    [p, href, https, en, wikipedia, org, wiki, per...
17    [p, stencil, buffer, unsigned, integer, bu

## Lemmatize

In [31]:
dfmini['body'] = dfmini['body'].apply(lambda x: lemmatizeText(x))

In [32]:
dfmini['body'].head()

0    [p, saw, href, http, www, youtube, com, watch,...
1    [p, convention, graphic, performing, fewer, st...
2    [p, sometimes, use, vector, graphic, simply, l...
3    [p, implemented, interpolated, texture, sampli...
4    [p, opengl, application, us, stencil, test, qu...
Name: body, dtype: object

### Number of different words in our data set:

In [47]:
tokens = dfmini['body']

vocabulary = set()
type(tokens)
print(tokens[1]) # example of a message body.


for items in tokens:
    for word in items:
        vocabulary.add(word)
    
print(len(vocabulary)) # We have this many words.

['p', 'like', 'suggestion', 'optimizing', 'brute', 'force', 'solution', 'href', 'http', 'projecteuler', 'net', 'index', 'php', 'section', 'problem', 'amp', 'id', '1', 'problem', '1', 'algorithm', 'currently', 'check', 'every', 'integer', '3', '1000', 'like', 'cut', 'many', 'unnecessary', 'call', 'code', 'ismultiple', 'code', 'possible', 'p', 'pre', 'code', 'list', 'natural', 'number', '10', 'multiple', '3', '5', 'get', '3', '5', '6', '9', 'sum', 'multiple', '23', 'find', 'sum', 'multiple', '3', '5', '1000', 'end', '1000', 'def', 'solution01', 'solved', 'brute', 'force', 'optimize', 'sum', '0', 'range', '3', 'end', 'ismultiple', 'sum', 'print', 'sum', 'def', 'ismultiple', 'return', '3', '0', '5', '0', 'code', 'pre']
756900


## TF-IDF

In [132]:
from sklearn.feature_extraction.text import TfidfVectorizer

questions = posts[pd.to_numeric(posts['PostTypeId']) == 1]['Body']
answers = posts[pd.to_numeric(posts['PostTypeId']) == 2]['Body']

data = list(set(questions))
for idx in range(len(data)):
    data[idx] = removePunctuation(removeHTML(removeURLs(removeTags(data[idx]))))

tfid_vectorizer = TfidfVectorizer()
tfid_vectorizer.fit_transform(data)
feature_names = tfid_vectorizer.get_feature_names()

In [133]:
feature_names

['00',
 '000',
 '0000',
 '00000',
 '000000',
 '0000000000',
 '000000000001',
 '00000001f',
 '0000001',
 '0000001f',
 '000001',
 '000001f',
 '00000390',
 '00001',
 '00001f',
 '0000200220022',
 '000041697',
 '00005',
 '00005111',
 '00006',
 '00007',
 '0001',
 '00010',
 '00016',
 '0001647949220t24955745390t962717676',
 '0001f',
 '00020',
 '000300',
 '000329589844t2',
 '0004',
 '0005',
 '000500',
 '00051',
 '0005f',
 '00075f',
 '000frac500',
 '001',
 '0010',
 '001000',
 '00107',
 '0016',
 '00175',
 '0018',
 '0019201080',
 '001cz',
 '001f',
 '002',
 '0020',
 '00201',
 '002210505050515151515',
 '0025',
 '00255',
 '0025f',
 '002gb',
 '003',
 '0030',
 '00312f',
 '00337',
 '0036742',
 '003f',
 '004',
 '00415',
 '0041556',
 '004257999496004257999496004257999496',
 '0044',
 '004472136089442719',
 '0045',
 '0047',
 '0048',
 '0049',
 '004f',
 '005',
 '0051',
 '005422300426920041095',
 '0055648',
 '00557',
 '0057',
 '005f',
 '005plot',
 '006',
 '00608',
 '006e5',
 '006f',
 '007',
 '00702',
 '0072',
